# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5NDc0ZmNhNi0wODFlLTRhYTktYjgwZS01MWJkMDMxNWJhNTAifQ=="
os.environ['NEPTUNE_PROJECT']="guilhermemg/icao-nets-training-2"
os.environ['NEPTUNE_NOTEBOOK_ID']="98a391a1-c710-40bd-aaf4-42c31862cbbe"
os.environ['NEPTUNE_NOTEBOOK_PATH']="training/exec_nas_experiment.ipynb"

In [3]:
import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

import sys
import pandas as pd

if '..' not in sys.path:
    sys.path.insert(0, '..')

In [4]:
from src.m_utils import constants as cts
from src.base.data_loaders.data_loader import DLName
from src.base.gt_loaders.gt_names import GTName
from src.exp_runner import ExperimentRunner

from src.base.experiment.dataset.dataset import Dataset
from src.base.experiment.evaluation.model_evaluator import DataSource, DataPredSelection
from src.base.experiment.training.base_models import BaseModel
from src.base.experiment.training.optimizers import Optimizer

from src.m_utils.stl_approach import STLApproach
from src.m_utils.mtl_approach import MTLApproach
from src.m_utils.nas_mtl_approach import NAS_MTLApproach

from src.nas.v3.nas_algorithm import NASAlgorithm
from src.nas.v3.mlp_search_space import MLPSearchSpaceIndicator

 ==> Restrict GPU memory growth: True


# Start Network Runner

In [21]:
DATASET = Dataset.MNIST
APPROACH = NAS_MTLApproach.APPROACH_3

kwargs = { 
    'use_neptune': False,
    'exp_params' : {
        'name': 'NAS experiment',
        'description': 'NAS with Approach 3 - Testing parametrization of n_convs',
        'tags': [f'{DATASET.value["name"]}', 'nas', 'nas_approach_3_v2', 'no_use_predictor'],
        'src_files': ["../src/**/*.py"]
    },
    'properties': {
        'approach': APPROACH,
        'dataset': DATASET,
        'tasks': DATASET.value['tasks'],
        'balance_input_data': False,
        'train_model': True,
        'save_trained_model': True,
        'exec_nas': True,
        'orig_model_experiment_id': '',
        'sample_training_data': True,
        'sample_prop': 0.02
    },
    'nas_params': {
        'architecture_training_epochs': 2,     # n_epochs for training proposed architecture
        'total_num_proposed_architectures': 10,
        'nas_algorithm': NASAlgorithm.RL,
        'nas_search_space': MLPSearchSpaceIndicator.SS_2
    },
    'controller_params': {
        'controller_lstm_dim': 100,
        'controller_optimizer': Optimizer.ADAM,
        'controller_learning_rate': 0.01,
        'controller_decay': 0.1,
        'controller_momentum': 0.0,
        'controller_use_predictor': False,
        'controller_loss_alpha': 0.9,
        'controller_training_epochs': 5,
        'controller_sampling_epochs': 6,
        'controller_batch_size': 3
    },
    'mlp_params': {
        # 'max_architecture_length': 5,
        'min_task_group_size': 3,
        'mlp_base_model': BaseModel.MOBILENET_V2,
        'mlp_n_epochs': 50,
        'mlp_batch_size': 64,
        'mlp_early_stopping': 50,
        'mlp_optimizer': Optimizer.ADAMAX,
        'mlp_learning_rate': 1e-3,
        'mlp_decay': 0.0,
        'mlp_momentum': 0.0,
        'mlp_dropout': 0.3,
        'mlp_loss_function': 'sparse_categorical_crossentropy',
        'mlp_one_shot': False
    }
}


runner = ExperimentRunner(**kwargs)

-------------------- Init ExperimentRunner -------------------
---------------------------
Parent Process ID: 115681
Process ID: 318361
---------------------------
-----
Use Neptune:  False
-----
-------------------
Args: 
{'controller_params': {'controller_batch_size': 3,
                       'controller_decay': 0.1,
                       'controller_learning_rate': 0.01,
                       'controller_loss_alpha': 0.9,
                       'controller_lstm_dim': 100,
                       'controller_momentum': 0.0,
                       'controller_optimizer': <Optimizer.ADAM: 'Adam'>,
                       'controller_sampling_epochs': 6,
                       'controller_training_epochs': 5,
                       'controller_use_predictor': False},
 'exp_params': {'description': 'NAS with Approach 3 - Testing parametrization '
                               'of n_convs',
                'name': 'NAS experiment',
                'src_files': ['../src/**/*.py'],
      

# Load Data

In [22]:
runner.load_training_data()

-------------------- load training data -------------------
Loading data
TrainData.shape: (48000, 11)
ValidationData.shape: (12000, 11)
TestData.shape: (10000, 11)
Data loaded


In [23]:
runner.sample_training_data()

-------------------- sample training data -------------------
Applying subsampling in training data
..Sampling proportion: 0.02 (960/48000)
(960, 11)
Applying subsampling in validation data
..Sampling proportion: 0.02 (240/12000)
(240, 11)


In [24]:
runner.train_data.head()

,img_name,n_0,n_1,n_2,n_3,n_4,n_5,n_6,n_7,n_8,n_9
32771,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
39512,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
43581,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2975,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3377,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


runner.produce_fake_data()

# Data Generators

In [25]:
runner.setup_data_generators()

-------------------- setup data generators -------------------
Starting data generators
Found 960 validated image filenames.
Found 240 validated image filenames.
Found 10000 validated image filenames.
TOTAL: 11200

Logging class indices
 .. MTL model not logging class indices!

Using benchmarking dataset. Not logging class labels!


# Setup Experiment

In [26]:
runner.setup_experiment()

-------------------- create experiment -------------------
Not using Neptune


# Labels Distribution

In [27]:
%%capture
runner.summary_labels_dist()

# Neural Architecture Search - v3

In [36]:
best_arch = runner.run_neural_architecture_search_v3()

-------------------- run neural architecture search -------------------
 -- Using search space: {'name': 'ss_2', 'n_classes': 8}
 -- Using NAS algorithm: NASAlgorithm.RL

Controller model input shape: (None, 1, 12)
Controller model output shape: (None, 1, 8)
Model path:  trained_model
----
Checking model existence locally...
Training a new model! Not checking model existence
----
------------------------------
Checking GPU availability
 ..GPU is available!
------------------------------
 ..prev_arch: [[[5 3 4 3 4 2 2 2]]]
 ..prev_arch.shape: (1, 1, 8)
input: [[[5 3 4 3 4 2 2 2]]]
noise: [[[3 5 3 7]]]
prob_list.shape: (1, 1, 8)
prob_list: [0.19503881 0.11027795 0.16114159 0.13942319 0.08442298 0.09571441
 0.08673017 0.12725094]
len(prob_list): 8
prob_list[:5]: [0.19503881 0.11027795 0.16114159 0.13942319 0.08442298]
len(prob_list[:5]): 5
chose_idx: 0 | lookup_search_space_candidates[chose_idx]: 1 | lookup_search_space_candidates: [1, 2, 3, 4, 5]
final_arch: [0]
(1, 8)
new input: [[[ 0 -

/home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/scipy/interpolate/interpolate.py:630: RuntimeWarning: divide by zero encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/scipy/interpolate/interpolate.py:633: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


  Task  0: n_0             | EER_mean: 0.09260 | EER_interp: 0.08800 | ACC: 0.90830
  Task  1: n_1             | EER_mean: 0.00000 | EER_interp: 0.00000 | ACC: 0.14170
  Task  2: n_2             | EER_mean: 0.39170 | EER_interp: 0.39150 | ACC: 0.60830
  Task  3: n_3             | EER_mean: 0.28570 | EER_interp: 0.28660 | ACC: 0.71250
  Task  4: n_4             | EER_mean: 0.42860 | EER_interp: 0.42890 | ACC: 0.57080
  Task  5: n_5             | EER_mean: 0.39550 | EER_interp: 0.39780 | ACC: 0.60420
  Task  6: n_6             | EER_mean: 0.56070 | EER_interp: 0.56880 | ACC: 0.43750
  Task  7: n_7             | EER_mean: 0.24770 | EER_interp: 0.26280 | ACC: 0.75000
  Task  8: n_8             | EER_mean: 0.35550 | EER_interp: 0.35010 | ACC: 0.64580
  Task  9: n_9             | EER_mean: 0.11110 | EER_interp: 0.11800 | ACC: 0.88750
final_EER_mean: 28.92% | final_EER_median: 31.84% | final_EER_std_dv: 16.67% | final_ACC: 62.67%
------------------------------------------------------------
 .

# Create Model with Best Architecture Found

In [ ]:
runner.create_model(best_arch)

# Visualize Model

In [ ]:
%%capture
runner.visualize_model(outfile_path=f"training/figs/nas/nas_model_{APPROACH.name}.png")

In [ ]:
%%capture
runner.model_summary()

# Training Model

In [ ]:
runner.train_model()

# Plots

In [ ]:
runner.draw_training_history()

# Load Best Model

In [ ]:
runner.load_best_model()

# Saving Trained Model

In [ ]:
runner.save_model()

# Test Trained Model

## Validation Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.VALIDATION)
runner.test_model(verbose=False)

## Test Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.TEST)
runner.test_model(verbose=False)

# Visualize Model Classification

# Finishing Experiment Manager

In [ ]:
runner.finish_experiment()